In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/store-transaction-data/Hackathon_Working_Data.csv')

In [ ]:
print('There are {} distinct data at Group level'.format(data['GRP'].nunique()))
print('There are {} distinct data at sub-Group level'.format(data['SGRP'].nunique()))
print('There are {} distinct data at sub-sub-Group level'.format(data['SSGRP'].nunique()))
print('There are {} distinct data at Company level'.format(data['CMP'].nunique()))
print('There are {} distinct data at Mother-Brand level'.format(data['MBRD'].nunique()))
print('There are {} distinct data at Brand level'.format(data['BRD'].nunique()))

### Data for Store N1

In [ ]:
data_N1 = data[data['STORECODE'] == 'N1']
print(data_N1.shape)

In [ ]:
data_N1 = data_N1[['BILL_ID', 'QTY', 'GRP']]

In [ ]:
basket = data_N1.groupby(['BILL_ID', 'GRP'])['QTY'].sum().unstack().reset_index().fillna(0).set_index('BILL_ID')

In [ ]:
def encode_unit(x):
    if x <= 0:
        return 0
    if x >= 0:
        return 1
    
encoded_data = basket.applymap(encode_unit)

In [ ]:
frequent_items = apriori(encoded_data, min_support = 0.01, use_colnames = True)

In [ ]:
rules = association_rules(frequent_items, metric = 'lift', min_threshold = 0)

### Automating Storewise Data

In [ ]:
store = list(data['STORECODE'].unique())
store

In [ ]:
for i in range(0,len(store)):
    data_N = data[data['STORECODE'] == store[i]]
    data_N = data_N[['BILL_ID', 'QTY', 'GRP']]
    
    basket = data_N.groupby(['BILL_ID', 'GRP'])['QTY'].sum().unstack().reset_index().fillna(0).set_index('BILL_ID')
    
    def encode_unit(x):
        if x <= 0:
            return 0
        if x >= 0:
            return 1
    
    encoded_data = basket.applymap(encode_unit)
    
    frequent_items = apriori(encoded_data, min_support = 0.01, use_colnames = True)
    
    rules = association_rules(frequent_items, metric = 'lift', min_threshold = 0)
    
    rules.to_csv('AssociationRuleforstore{}.csv'.format(store[i]))